### Instapaper

In [7]:
import re
# from dateutil import parser
import pandas as pd
# from dotenv import load_dotenv
# import os

import requests
from bs4 import BeautifulSoup as bs
import csv
import time
#time.sleep(2)

In [8]:
pwd

'/Users/edwardterrell/Desktop/Training/kindle/instapaper'

In [9]:
# It prints the name of the current Git branch you’re on.
!git rev-parse --abbrev-ref HEAD

insta_test


In [10]:
import sys
sys.executable

'/Users/edwardterrell/opt/anaconda3/envs/scrape/bin/python'

In [11]:
!pwd

/Users/edwardterrell/Desktop/Training/kindle/instapaper


In [12]:
# Accept the url and output the table for a state
# url = 'https://www.instapaper.com'
# response = requests.get(url)
# print(response.status_code)    #200 = success
# print(response.text[:200])   #First 200 characters of the html string
# page = response.text 
# soup = bs(page)
# soup

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [14]:
from pathlib import Path
base_path = Path.home() / "bin" / "passwords"
with open(base_path / "instapaper_username.txt", "r") as f:
    username = f.read().strip()
with open(base_path / "instapaper_password.txt", "r") as f:
    password = f.read().strip()

#print(username, password)  # Just for testing

In [15]:
# Start Safari (or Chrome)
driver = webdriver.Safari()

driver.get("https://www.instapaper.com/user/login")
username_field = driver.find_element(By.ID, "username")
password_field = driver.find_element(By.ID, "password")

# Fill in login form
username_field.send_keys(username)
time.sleep(1)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)

# Wait for redirect
time.sleep(3)

def parse_url(url):
    # Go to archive and grab links
    driver.get(url)
    # Instead of time.sleep(2) we use WebDriverWait
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'article_text_content'))
        )
        html = driver.page_source
    except TimeoutException:
        print("Timeout: The page didn't load the expcted content.")
    
    # Now we are in the beautiful soup world
    soup = bs(html, "html.parser")
    links = [a['href'] for a in soup.find_all('a', href=True)]  
    clean_links = [link for link in links if link.startswith("http")]
    return clean_links, soup  # <-- this was missing earlier

# assign the function's return variables to be used out of scope
url = "https://www.instapaper.com/archive"
clean_links, soup = parse_url(url)   

driver.quit()
# 
print(len(clean_links))
len(soup)

50


1

In [16]:
# ??preview = preview_div.text.strip()


In [17]:
# This creates a file: 'all_soup.html'
# by opening this file in terminal, a browser window opens which you can inspect as if it were live
with open("all_soup.html", "w", encoding="utf-8") as f:
    f.write(soup.prettify())

In [18]:
# Generates a txt file with urls - one per line
# FILENAME: filtered_links.txt

import csv

# instapaper_clean_links is a temp file; only to create filtered_links(the variable)
with open("instapaper_clean_links.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["URL"])
    for url in clean_links:
        writer.writerow([url])  # wrap url in a list!

# Filter out links containing "instapaper" or "ads/click"
filtered_links = [
    url for url in clean_links
    if 'instapaper' not in url and 'ads/click' not in url
]

# Save to a plain text file, one URL per line
with open("filtered_links.txt", "w", encoding="utf-8") as f:
    for url in filtered_links:
        f.write(url + "\n")

In [19]:
# Convert plain text URLs to a clickable HTML file:
# FILENAME: clickable_links.html

# Open the input file with your filtered links
with open("filtered_links.txt", "r") as infile:
    urls = [line.strip() for line in infile if line.strip()]

# Create an HTML file with clickable links
with open("clickable_links.html", "w") as outfile:
    outfile.write("<!DOCTYPE html>\n<html>\n<body>\n<h2>Clickable Links</h2>\n<ul>\n")
    for url in urls:
        outfile.write(f'  <li><a href="{url}" target="_blank">{url}</a></li>\n')
    outfile.write("</ul>\n</body>\n</html>")


### links only program ends here

In [20]:
archive_link = soup.find('a', class_='paginate_older')['href']
archive_link

'https://www.instapaper.com/archive/2'

### titles,previews,urls for df2

In [21]:
# titles only
titles= [title.text.strip().replace('\xa0',' ') for title in soup.find_all('a', class_='article_title')]
# previews only
previews = [prev.text.strip().replace('\xa0',' ') for prev in soup.find_all('div', class_='article_preview')]
# url links only
url_links = [link['href'] for link in soup.find_all('a', class_='js_domain_linkout')]

In [22]:
# these are clickable links
len(url_links)

40

In [23]:
df2 = pd.DataFrame({'Title':titles,'Link':url_links,'Preview':previews})
df2.head()

,Title,Link,Preview
0,Paradox Articles and Information,http://neohumanism.org/p/pa/paradox.html,
1,Goedel's Theorem and Information,http://www.cs.auckland.ac.nz/~chaitin/georgia....,Gödel's Theorem and Information International ...
2,John von Neumann’s The Computer and the Brain,http://www.leydesdorff.net/vonneumann/,return The Code of Mathematics John von Neuman...
3,US wealth gap between young and old is widest ...,http://news.yahoo.com/us-wealth-gap-between-yo...,Our team is dedicated to finding and telling y...
4,Computing Machinery and Intelligence A.M. Turing,http://www.loebner.net/Prizef/TuringArticle.html,Latest News Influencer Marketing The Advantage...


In [24]:
article_links = []

# Loop through all article title elements
# A less convulted approach might be to use three for loops above

for title_tag in soup.find_all('a', class_='article_title'):
    title = title_tag.text.strip().replace('\xa0', ' ')
    
    # Look for the title's parent parent div
    title_div = title_tag.parent.parent
    link_div = title_div.find_next_sibling('div')  # skips whitespace, finds actual div
    preview_div = link_div.find_next_sibling('div')
    preview = preview_div.text.strip()

    url = link_div.find('a')['href']
    article_links.append((title, url, preview))
    #print(preview)

In [25]:
# article_links

In [26]:
# Creates a csv file of the results
with open("instapaper_articles.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Title", "URL", "Preview"])
    writer.writerows(article_links)

In [27]:
# Pandas version
df = pd.read_csv('instapaper_articles.csv',na_filter=False)
df.head()

,Title,URL,Preview
0,Paradox Articles and Information,http://neohumanism.org/p/pa/paradox.html,
1,Goedel's Theorem and Information,http://www.cs.auckland.ac.nz/~chaitin/georgia....,Gödel's Theorem and Information International ...
2,John von Neumann’s The Computer and the Brain,http://www.leydesdorff.net/vonneumann/,return The Code of Mathematics John von Neuman...
3,US wealth gap between young and old is widest ...,http://news.yahoo.com/us-wealth-gap-between-yo...,Our team is dedicated to finding and telling y...
4,Computing Machinery and Intelligence A.M. Turing,http://www.loebner.net/Prizef/TuringArticle.html,Latest News Influencer Marketing The Advantage...


In [56]:
df.iloc[1,2]

"Gödel's Theorem and Information International Journal of Theoretical Physics 21 (1982), pp.941-954 Gregory J. Chaitin IBM Research, P.O. Box 218 Yorktown Heights, New York 10598 Abstract Gödel's theorem may be demonstrated using arguments having aninformation-theoretic flavor. In such an approach it is possibleto argue that if a theorem contains more information than a givenset of axioms, then it…"

In [28]:
from urllib.parse import urlparse
# Read the CSV and avoid NaN
df = pd.read_csv("instapaper_articles.csv", na_filter=False)

### Store in an html file for web viewing and clickable links

In [29]:
df.URL.head()

0             http://neohumanism.org/p/pa/paradox.html
1    http://www.cs.auckland.ac.nz/~chaitin/georgia....
2               http://www.leydesdorff.net/vonneumann/
3    http://news.yahoo.com/us-wealth-gap-between-yo...
4     http://www.loebner.net/Prizef/TuringArticle.html
Name: URL, dtype: object

In [30]:
# Convert URL column to clickable HTML links
# df['URL'] = df['URL'].apply(lambda x: f'<a href="{x}" target="_blank">{x}</a>')

df['URL'] = df['URL'].apply(
    lambda x: f'<a href="{x}" target="_blank">{urlparse(x).netloc}</a>'
)

In [33]:
df.head()

,Title,URL,Preview
0,Paradox Articles and Information,"<a href=""http://neohumanism.org/p/pa/paradox.h...",
1,Goedel's Theorem and Information,"<a href=""http://www.cs.auckland.ac.nz/~chaitin...",Gödel's Theorem and Information International ...
2,John von Neumann’s The Computer and the Brain,"<a href=""http://www.leydesdorff.net/vonneumann...",return The Code of Mathematics John von Neuman...
3,US wealth gap between young and old is widest ...,"<a href=""http://news.yahoo.com/us-wealth-gap-b...",Our team is dedicated to finding and telling y...
4,Computing Machinery and Intelligence A.M. Turing,"<a href=""http://www.loebner.net/Prizef/TuringA...",Latest News Influencer Marketing The Advantage...


In [32]:
# Export to HTML
df.to_html("Final_clickable_links.html", escape=False, index=False)

In [ ]:
# Export to HTML
html_table = df.to_html(escape=False, index=False)

html_full = f"""<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Exported Table</title>
</head>
<body>
{html_table}
</body>
</html>
"""

with open("article_listing.html", "w", encoding="utf-8") as f:
    f.write(html_full)


### Info on datetime

In [131]:
import dateutil.parser 
from dateutil import parser
# MORE FORGIVING
# date_admitted = dateutil.parser.parse(date_admitted_str)
dt = parser.parse("June 1st, 2025 2:30 PM")
print(dt)
dt

2025-06-01 14:30:00


datetime.datetime(2025, 6, 1, 14, 30)

In [135]:
# MUST KNOW the expected format
from datetime import datetime
date_str = "June 1, 2025"
dt_date = datetime.strptime(date_str, "%B %d, %Y")
dt_date.year

2025

In [ ]:
soup.find_all('div',class_ ='title_meta')

soup.find_all(class_ ='title_meta')

soup.find_all('a',class_='js_domain_linkout')

today_links = [link['href'] for link in today_div.find_all('a')]

### Using regex and the DOM

In [ ]:
adm = re.compile('Admitted')
soup.find(text=adm)
# This looks like a string, but it's actually a BeautifulSoup element.
# type(soup.find(text=regex))
# bs4.element.NavigableString

In [ ]:
adm.next.text
adm.parent

In [ ]:
def to_date(date_str):
    date_str = re.match('[\w]+\s[\d,\s]+', date_str)[0]
    return dateutil.parser.parse(date_str)

def to_int(number_str):
    number_str = re.match('[\d,$]+', number_str)[0]
    number_str = number_str.replace('$', '').replace(',', '')
    return int(number_str)

In [ ]:
re.match('[\d,$]+', area_text)[0]

In [ ]:
### Plan the structure of what your going to save and how

In [ ]:
penn_dict = {
    'state': state,
    'date_admitted': date_admitted,
    'population': population,
    'area_sq_mi': area
}

In [ ]:
penn_info = [penn_dict]
penn_df = pd.DataFrame(penn_info)
penn_df

In [ ]:
penn_df.to_csv('Penn_State_Information.csv')

In [ ]:
# to add a col to a df
penn_df['median_income'] = m

### save as functions so that they can be reused

In [ ]:
# Accept the url and output the table for a state
ny_url = 'https://en.wikipedia.org/wiki/New_York_(state)'
ny_page = requests.get(ny_url).text
ny_soup = bs(ny_page)

ny_table = ny_soup.find('table')
#ny_table

### Data Storaage

In [ ]:
df.to_csv('temp',index=False)
df.to_excel("data.xlsx", index=False)

df.to_hdf("data.h5", key="my_data", mode="w" )

import sqlite3
conn = sqlite3.connect("temp.db")
df.to_sql("my_table", conn, if_exists="replace", index=False)

import pickle
with open("temp.pkl", "wb") as file:
    pickle.dump(df, file)

# Feather is a lightweight and fast binary columnar storage format
import feather
feather.write_dataframe(df, "temp.feather")

### Other

In [35]:
# Example of basic login script with selenium
from selenium import webdriver

# This uses the system SafariDriver
driver = webdriver.Safari()

driver.get("https://example.com")
html = driver.page_source

# You can use BeautifulSoup here if needed
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
links = [a['href'] for a in soup.find_all('a', href=True)]

print(links)


['https://www.iana.org/domains/example']
